# Brain Networks in Python

scona is a tool to perform network analysis over correlation networks of brain regions. 
This tutorial will go through the basic functionality of scona, taking us from our inputs (a matrix of structural regional measures over subjects) to a report of local network measures for each brain region, and network level comparisons to a cohort of random graphs of the same degree. 

In [2]:
import numpy as np
import networkx as nx
import scona as scn
import scona.datasets as datasets

### Importing data

A scona analysis starts with four inputs.
* __regional_measures__
    A pandas DataFrame with subjects as rows. The columns should include structural measures for each brain region, as well as any subject-wise covariates. 
* __names__
    A list of names of the brain regions. This will be used to specify which columns of the __regional_measures__ matrix to want to correlate over.
* __covars__ _(optional)_ 
    A list of your covariates. This will be used to specify which columns of __regional_measure__ you wish to correct for. 
* __centroids__
    A list of tuples representing the cartesian coordinates of brain regions. This list should be in the same order as the list of brain regions to accurately assign coordinates to regions. The coordinates are expected to obey the convention the the x=0 plane is the same plane that separates the left and right hemispheres of the brain. 

In [3]:
# Read in sample data from the NSPN WhitakerVertes PNAS 2016 paper.
df, names, covars, centroids = datasets.NSPN_WhitakerVertes_PNAS2016.import_data()

In [4]:
df.head()

Unnamed: 0  nspn_id  occ     centre study_primary  age_scan     sex  male  \
0           0    10356    0  Cambridge     2K_Cohort    20.761  Female   0.0   
1           1    10702    0  Cambridge     2K_Cohort    16.055    Male   1.0   
2           2    10736    0  Cambridge     2K_Cohort    14.897  Female   0.0   
3           3    10778    0  Cambridge     2K_Cohort    20.022  Female   0.0   
4           4    10794    0  Cambridge     2K_Cohort    14.656  Female   0.0   

   age_bin mri_centre       ...         rh_supramarginal_part5  \
0        4       WBIC       ...                          2.592   
1        2       WBIC       ...                          3.448   
2        1       WBIC       ...                          3.526   
3        4       WBIC       ...                          2.830   
4        1       WBIC       ...                          2.689   

   rh_supramarginal_part6  rh_supramarginal_part7  rh_frontalpole_part1  \
0                   2.841                   2.318                 2.486   
1                   3.283                   2.740                 3.225   
2                   3.269                   3.076                 3.133   
3                   2.917                   2.647                 2.796   
4                   3.294                   2.820                 2.539   

   rh_temporalpole_part1  rh_transversetemporal_part1  rh_insula_part1  \
0                  3.526                        2.638            3.308   
1                  4.044                        3.040            3.867   
2                  3.900                        2.914            3.894   
3                  3.401                        3.045            3.138   
4                  2.151                        2.734            2.791   

   rh_insula_part2  rh_insula_part3  rh_insula_part4  
0            2.583            3.188            3.089  
1            2.943            3.478            3.609  
2            2.898            3.720            3.580  
3            2.739            2.833            3.349  
4            2.935            3.538            3.403  

[5 rows x 324 columns]

### Create a correlation matrix
We calculate residuals of the matrix df for the columns of names, correcting for the columns in covars.

In [5]:
df_res = scn.create_residuals_df(df, names, covars)

In [6]:
df_res

lh_bankssts_part1  lh_bankssts_part2  lh_caudalanteriorcingulate_part1  \
0            -0.016677          -0.036253                          0.035892   
1             0.280323           0.354747                          0.482892   
2             0.168323           0.081747                          0.365892   
3            -0.086677          -0.249253                         -0.404108   
4             0.400323           0.136747                          0.125892   
5             0.010323          -0.090253                         -0.248108   
6            -0.118677           0.133747                          0.089892   
7             0.106323           0.043747                          0.355892   
8             0.433323           0.037747                          0.191892   
9             0.191323           0.081747                          0.093892   
10           -1.036677          -0.759253                         -0.621108   
11            0.183323           0.155747                         -0.100108   
12           -0.058677          -0.222253                         -0.230108   
13            0.200323          -0.048253                          0.032892   
14            0.221323           0.374747                          0.107892   
15            0.075323          -0.141253                         -0.161108   
16           -0.047677           0.226747                         -0.067108   
17            0.055323          -0.055253                          0.164892   
18           -0.327677           0.128747                         -0.260108   
19           -0.012677          -0.090253                          0.499892   
20            0.052323          -0.096253                          0.879892   
21           -0.028677          -0.236253                         -0.091108   
22            0.235323           0.102747                          0.235892   
23            0.120323           0.066747                         -0.144108   
24            0.084323           0.094747                         -0.203108   
25            0.098323           0.030747                          0.304892   
26            0.024323          -0.112253                          0.335892   
27            0.215323          -0.008253                         -0.041108   
28           -0.170677          -0.324253                         -0.035108   
29           -0.102677          -0.035253                         -0.117108   
..                 ...                ...                               ...   
267          -0.604677          -0.283253                         -0.478108   
268          -0.090677          -0.174253                         -0.005108   
269          -0.357677          -0.119253                          0.010892   
270           0.166323           0.225747                          0.357892   
271          -0.056677           0.032747                          0.074892   
272          -0.159677          -0.130253                         -0.410108   
273          -0.044677          -0.066253                         -0.054108   
274           0.591323           0.204747                          0.964892   
275           0.290323           0.104747                          0.129892   
276          -0.197677          -0.123253                         -0.140108   
277          -0.115677          -0.187253                         -0.163108   
278          -0.020677           0.037747                          0.184892   
279          -0.197677          -0.128253                         -0.123108   
280           0.040323           0.115747                         -0.322108   
281          -0.081677          -0.254253                         -0.144108   
282           0.304323           0.275747                          0.050892   
283          -0.305677          -0.238253                         -0.178108   
284          -0.027677          -0.158253                          0.325892   
285          -0.216677          -0.194253              

Now we create a correlation matrix over the columns of df_res

In [7]:
M = scn.create_corrmat(df_res, method='pearson')

## Create a weighted graph

A short sidenote on the BrainNetwork class: This is a very lightweight subclass of the [`Networkx.Graph`](https://networkx.github.io/documentation/stable/reference/classes/graph.html) class. This means that any methods you can use on a `Networkx.Graph` object can also be used on a `BrainNetwork` object, although the reverse is not true. We have added various methods which allow us to keep track of measures that have already been calculated, which, especially later on when one is dealing with 10^3 random graphs, saves a lot of time.  
All scona measures are implemented in such a way that they can be used on a regular `Networkx.Graph` object. For example, instead of `G.threshold(10)` you can use `scn.threshold_graph(G, 10)`.  
Also you can create a `BrainNetwork` from a `Networkx.Graph` `G`, using `scn.BrainNetwork(network=G)`

Initialise a weighted graph `G` from the correlation matrix `M`. The `parcellation` and `centroids` arguments are used to label nodes with names and coordinates respectively. 

In [8]:
G = scn.BrainNetwork(network=M, parcellation=names, centroids=centroids)

### Threshold to create a binary graph

We threshold G at cost 10 to create a binary graph with 10% as many edges as the complete graph G. Ordinarily when thresholding one takes the 10% of edges with the highest weight. In our case, because we want the resulting graph to be connected, we calculate a minimum spanning tree first. If you want to omit this step, you can pass the argument `mst=False` to `threshold`.
The threshold method does not edit objects inplace

In [9]:
H = G.threshold(10)

### Calculate nodal summary. 

`calculate_nodal_measures` will compute and record the following nodal measures 

* average_dist (if centroids available)
* total_dist (if centroids available)
* betweenness
* closeness
* clustering coefficient
* degree
* interhem (if centroids are available)
* interhem_proportion (if centroids are available)
* nodal partition
* participation coefficient under partition calculated above
* shortest_path_length

`export_nodal_measure` returns nodal attributes in a DataFrame. Let's try it now.

In [11]:
H.report_nodal_measures().head()

centroids                              name  \
0   [-27.965157, -19.013702, 17.919528]                 lh_bankssts_part1   
1   [-14.455663, -13.693461, 13.713674]                 lh_bankssts_part2   
2  [-33.906934, -22.284672, -15.821168]  lh_caudalanteriorcingulate_part1   
3  [-17.305373, -53.431573, -36.017154]      lh_caudalmiddlefrontal_part1   
4  [-22.265823, -64.366296, -37.674831]      lh_caudalmiddlefrontal_part2   

         x        y        z  
0 -27.9652 -19.0137  17.9195  
1 -14.4557 -13.6935  13.7137  
2 -33.9069 -22.2847 -15.8212  
3 -17.3054 -53.4316 -36.0172  
4 -22.2658 -64.3663 -37.6748

Use `calculate_nodal_measures` to fill in a bunch of nodal measures

In [12]:
H.calculate_nodal_measures()

        Calculating participation coefficient -           may take a little while


In [14]:
H.report_nodal_measures().head()

betweenness                             centroids closeness clustering  \
0  0.00824713   [-27.965157, -19.013702, 17.919528]  0.495961     0.3358   
1   0.0124798   [-14.455663, -13.693461, 13.713674]  0.507438   0.278788   
2           0  [-33.906934, -22.284672, -15.821168]  0.336254          1   
3   0.0120765  [-17.305373, -53.431573, -36.017154]  0.525685   0.383485   
4   0.0292617  [-22.265823, -64.366296, -37.674831]  0.549195   0.293617   

  degree module                              name participation_coefficient  \
0     47      0                 lh_bankssts_part1                  0.717067   
1     55      0                 lh_bankssts_part2                  0.809587   
2      2      1  lh_caudalanteriorcingulate_part1                      0.75   
3     83      2      lh_caudalmiddlefrontal_part1                  0.459864   
4     95      2      lh_caudalmiddlefrontal_part2                  0.688753   

  shortest_path_length        x        y        z  
0           0.00824713 -27.9652 -19.0137  17.9195  
1            0.0124798 -14.4557 -13.6935  13.7137  
2                    0 -33.9069 -22.2847 -15.8212  
3            0.0120765 -17.3054 -53.4316 -36.0172  
4            0.0292617 -22.2658 -64.3663 -37.6748

We can also add measures as one might normally add nodal attributes to a networkx graph

In [15]:
nx.set_node_attributes(H, name="hat", values={x: x**2 for x in H.nodes})

These show up in our DataFrame too

In [17]:
H.report_nodal_measures(columns=['name', 'degree', 'hat']).head()

degree hat                              name
0     47   0                 lh_bankssts_part1
1     55   1                 lh_bankssts_part2
2      2   4  lh_caudalanteriorcingulate_part1
3     83   9      lh_caudalmiddlefrontal_part1
4     95  16      lh_caudalmiddlefrontal_part2

### Calculate Global measures

In [18]:
H.calculate_global_measures()

{'assortativity': 0.09076922258276784,
 'average_clustering': 0.4498887255891581,
 'average_shortest_path_length': 2.376242649858285,
 'efficiency': 0.47983958611582617,
 'modularity': 0.3828553111606414}

In [20]:
H.rich_club();

## Create a GraphBundle

The `GraphBundle` object is the scona way to handle across network comparisons. What is it? Essentially it's a python dictionary with `BrainNetwork` objects as values. 

In [21]:
brain_bundle = scn.GraphBundle([H], ['NSPN_cost=10'])

This creates a dictionary-like object with BrainNetwork `H` keyed by `'NSPN_cost=10'`

In [22]:
brain_bundle

{'NSPN_cost=10': <scona.classes.BrainNetwork at 0x7f3f247f6fd0>}

Now add a series of random_graphs created by edge swap randomisation of H (keyed by `'NSPN_cost=10'`)

In [23]:
# Note that 10 is not usually a sufficient number of random graphs to do meaningful analysis,
# it is used here for time considerations
brain_bundle.create_random_graphs('NSPN_cost=10', 10)

        Creating 10 random graphs - may take a little while


In [24]:
brain_bundle

{'NSPN_cost=10': <scona.classes.BrainNetwork at 0x7f3f247f6fd0>,
 'NSPN_cost=10_R1': <scona.classes.BrainNetwork at 0x7f3f138b7f28>,
 'NSPN_cost=10_R10': <scona.classes.BrainNetwork at 0x7f3f134b8828>,
 'NSPN_cost=10_R2': <scona.classes.BrainNetwork at 0x7f3f138b7e80>,
 'NSPN_cost=10_R3': <scona.classes.BrainNetwork at 0x7f3f138b7780>,
 'NSPN_cost=10_R4': <scona.classes.BrainNetwork at 0x7f3f138b7da0>,
 'NSPN_cost=10_R5': <scona.classes.BrainNetwork at 0x7f3f1388ae10>,
 'NSPN_cost=10_R6': <scona.classes.BrainNetwork at 0x7f3f134b8a58>,
 'NSPN_cost=10_R7': <scona.classes.BrainNetwork at 0x7f3f134b84a8>,
 'NSPN_cost=10_R8': <scona.classes.BrainNetwork at 0x7f3f134b8390>,
 'NSPN_cost=10_R9': <scona.classes.BrainNetwork at 0x7f3f134b83c8>}

### Report on a GraphBundle

The following method will calculate global measures ( if they have not already been calculated) for all of the graphs in `graph_bundle` and report the results in a DataFrame. We can do the same for rich club coefficients below.

In [25]:
brain_bundle.report_global_measures()

assortativity  average_clustering  \
NSPN_cost=10           0.090769            0.449889   
NSPN_cost=10_R1       -0.083993            0.222571   
NSPN_cost=10_R10      -0.087652            0.222462   
NSPN_cost=10_R2       -0.080897            0.230384   
NSPN_cost=10_R3       -0.102626            0.221894   
NSPN_cost=10_R4       -0.072122            0.221924   
NSPN_cost=10_R5       -0.086952            0.232706   
NSPN_cost=10_R6       -0.084438            0.223727   
NSPN_cost=10_R7       -0.080285            0.224930   
NSPN_cost=10_R8       -0.070332            0.225730   
NSPN_cost=10_R9       -0.086510            0.227159   

                  average_shortest_path_length  efficiency  modularity  
NSPN_cost=10                          2.376243    0.479840    0.382855  
NSPN_cost=10_R1                       2.086721    0.519147    0.000000  
NSPN_cost=10_R10                      2.087821    0.518918    0.000000  
NSPN_cost=10_R2                       2.090423    0.518615    0.000000  
NSPN_cost=10_R3                       2.082745    0.519803    0.000000  
NSPN_cost=10_R4                       2.087842    0.518902    0.000000  
NSPN_cost=10_R5                       2.084754    0.519357    0.000000  
NSPN_cost=10_R6                       2.085325    0.519440    0.000000  
NSPN_cost=10_R7                       2.087525    0.519003    0.000000  
NSPN_cost=10_R8                       2.085135    0.519281    0.000000  
NSPN_cost=10_R9                       2.090782    0.518527    0.000000

In [26]:
brain_bundle.report_rich_club()

0         1         2         3         4         5    \
NSPN_cost=10      0.100004  0.103228  0.107244  0.112039  0.117842  0.122398   
NSPN_cost=10_R1   0.100004  0.103228  0.107175  0.111920  0.117564  0.121950   
NSPN_cost=10_R10  0.100004  0.103228  0.107175  0.111920  0.117564  0.121950   
NSPN_cost=10_R2   0.100004  0.103228  0.107175  0.111920  0.117564  0.121950   
NSPN_cost=10_R3   0.100004  0.103228  0.107175  0.111920  0.117589  0.122003   
NSPN_cost=10_R4   0.100004  0.103228  0.107175  0.111920  0.117564  0.121976   
NSPN_cost=10_R5   0.100004  0.103228  0.107175  0.111920  0.117564  0.121950   
NSPN_cost=10_R6   0.100004  0.103228  0.107175  0.111920  0.117564  0.121950   
NSPN_cost=10_R7   0.100004  0.103228  0.107175  0.111920  0.117564  0.121976   
NSPN_cost=10_R8   0.100004  0.103228  0.107175  0.111920  0.117564  0.121950   
NSPN_cost=10_R9   0.100004  0.103228  0.107175  0.111920  0.117564  0.121976   

                       6         7         8         9   ...   96   97   98   \
NSPN_cost=10      0.127975  0.131899  0.136820  0.141069 ...   1.0  1.0  1.0   
NSPN_cost=10_R1   0.127226  0.131092  0.135825  0.139877 ...   1.0  1.0  1.0   
NSPN_cost=10_R10  0.127226  0.131150  0.135885  0.139908 ...   0.0  0.0  0.0   
NSPN_cost=10_R2   0.127226  0.131092  0.135855  0.139971 ...   1.0  1.0  1.0   
NSPN_cost=10_R3   0.127282  0.131150  0.135885  0.139940 ...   1.0  1.0  1.0   
NSPN_cost=10_R4   0.127282  0.131150  0.135885  0.139940 ...   1.0  1.0  1.0   
NSPN_cost=10_R5   0.127254  0.131150  0.135915  0.139940 ...   1.0  1.0  1.0   
NSPN_cost=10_R6   0.127226  0.131092  0.135885  0.139908 ...   1.0  1.0  1.0   
NSPN_cost=10_R7   0.127254  0.131150  0.135915  0.139971 ...   1.0  1.0  1.0   
NSPN_cost=10_R8   0.127226  0.131121  0.135915  0.139940 ...   1.0  1.0  1.0   
NSPN_cost=10_R9   0.127254  0.131150  0.135915  0.140034 ...   0.0  0.0  0.0   

                  99   100  101  102  103  104  105  
NSPN_cost=10      1.0  1.0  1.0  1.0  1.0  1.0  1.0  
NSPN_cost=10_R1   1.0  1.0  1.0  1.0  1.0  1.0  1.0  
NSPN_cost=10_R10  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
NSPN_cost=10_R2   1.0  1.0  1.0  1.0  1.0  1.0  1.0  
NSPN_cost=10_R3   1.0  1.0  1.0  1.0  1.0  1.0  1.0  
NSPN_cost=10_R4   1.0  1.0  1.0  1.0  1.0  1.0  1.0  
NSPN_cost=10_R5   1.0  1.0  1.0  1.0  1.0  1.0  1.0  
NSPN_cost=10_R6   1.0  1.0  1.0  1.0  1.0  1.0  1.0  
NSPN_cost=10_R7   1.0  1.0  1.0  1.0  1.0  1.0  1.0  
NSPN_cost=10_R8   1.0  1.0  1.0  1.0  1.0  1.0  1.0  
NSPN_cost=10_R9   0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[11 rows x 106 columns]